**Homeworks – Graph Theory and Algorithms, 2021/22. semester I.**

Week 4

- name: Nagy, Balázs
- neptun: hpq7oi
- e-mail: [balazs_nagy@outlook.com](balazs_nagy@outlook.com)

![img/GTA_problem_set_4.png](img/GTA_problem_set_4.png)

First, I will implement the solution for excercise 2 and then I will use excercise 1 as an example to demonstrate it.

# Validation rules

The user can define the dictionaries `men_pref` and `women_pref`.

Requirements on the input:
1. the intersection of the `keys` of `men_pref` and `women_pref` should be an empty set;
2. the datatype of the `values` of `men_pref` and `women_pref` dictionaries should be `list`;
3. both dictionaries should pass the validation defined in section 1.1


## Intersection of keys mus be an empty set

In [1]:
def unique_keys(list_of_dictionaries:list):
    list_of_sets = [set(dictionary.keys()) for dictionary in list_of_dictionaries]
    assert len(set.intersection(*list_of_sets)) == 0, 'The intersection of the set of the keys of the dictionaries is not an empty set.'

Valid inputs:

In [2]:
men_pref = {0: [5,6,7], 1: [6,4,7], 2: [5,7,4], 3: [6,5,4]}
women_pref = {4: [1,2,3], 5: [2,3,0], 6: [1,3,0], 7: [2,0,1]}
list_of_dictionaries= [men_pref, women_pref]
unique_keys(list_of_dictionaries)

Invalid inputs:

In [3]:
men_pref = {0: [5,6,7], 1: [6,4,7], 2: [5,7,4], 3: [6,5,4]}
women_pref = {3: [1,2,3], 5: [2,3,0], 6: [1,3,0], 7: [2,0,1]}
list_of_dictionaries= [men_pref, women_pref]
unique_keys(list_of_dictionaries)

AssertionError: The intersection of the set of the keys of the dictionaries is not an empty set.

## Values of the dict must be lists

In [4]:
def values_are_lists(dictionary):
    for key, value in dictionary.items():
        assert isinstance(value, list), f'Data type of the values must be lists. key={key}: type of value={type(value)}'

Valid inputs:

In [5]:
men_pref = {0: [5,6,7], 1: [6,4,7], 2: [5,7,4], 3: [6,5,4]}
values_are_lists(men_pref)

Invalid inputs:

In [6]:
men_pref_invalid = {0: [5,6,7], 1: (6,4,7), 2: [5,7,4], 3: [6,5,4]}
values_are_lists(men_pref_invalid)

AssertionError: Data type of the values must be lists. key=1: type of value=<class 'tuple'>

## Inputs define a unique graph

>  Define a "compatibility" function that returns the string `Warning: incompatible lists` if the two lists of
preference lists define diffeerent graphs. This is much nicer than getting a list-index error message.

In [7]:
def compatibility(self_dict, reference_dict, print_on=False):
    # empty list to store results
    results = []
    compatible = True
    
    # check if we are in the list of the people we like
    # if yes -> return True
    # if no -> raise ValueError
    
    for we, people_we_like in self_dict.items():
        if print_on:
            print(f'we={we} and our list={people_we_like}')
        for person_we_like in people_we_like:
            we_are_in_the_list_of_people_we_like = we in reference_dict[person_we_like]
            if print_on:
                print(f'  {we_are_in_the_list_of_people_we_like}: person we like={person_we_like} list={reference_dict[person_we_like]}')
            if we_are_in_the_list_of_people_we_like:
                results.append(we_are_in_the_list_of_people_we_like)
            else:
                compatible = False
    assert compatible == True, f'Warning: incompatible lists. {we} is not in the list of person={person_we_like} (persons list={reference_dict[person_we_like]})'

Testing the function for valid items.

In [8]:
men_pref = {0: [5,6,7], 1: [6,4,7], 2: [5,7,4], 3: [6,5,4]}
women_pref = {4: [1,2,3], 5: [2,3,0], 6: [1,3,0], 7: [2,0,1]}

In [9]:
compatibility(self_dict=men_pref, reference_dict=women_pref)

In [10]:
compatibility(self_dict=women_pref, reference_dict=men_pref)

Testing the function for invalid items ($7$ knows $0$ but $0$ claims not knowing $7$).

In [11]:
men_pref_invalid = {0: [5,6], 1: [6,4,7], 2: [5,7,4], 3: [6,5,4]}
women_pref_invalid = {4: [1,2,3], 5: [2,3,0], 6: [1,3,0], 7: [2,0,1]}

In [12]:
compatibility(self_dict=men_pref_invalid, reference_dict=women_pref_invalid)

In [13]:
compatibility(self_dict=women_pref_invalid, reference_dict=men_pref_invalid)

AssertionError: Warning: incompatible lists. 7 is not in the list of person=1 (persons list=[6, 4, 7])

## Gather validation rules

As a reminder, the following needs to be checked:

The user can define the dictionaries `men_pref` and `women_pref`.

Requirements on the input:
1. the intersection of the `keys` of `men_pref` and `women_pref` should be an empty set;
2. the datatype of the `values` of `men_pref` and `women_pref` dictionaries should be `list`;
3. both dictionaries should pass the validation defined in section 1.1

In [14]:
def validate_inputs(dict1, dict2):
    # Assertion rule1
    unique_keys([dict1, dict2])
    
    # Assertion rule2
    values_are_lists(dict1)
    values_are_lists(dict2)
    
    # Assertion rule3
    compatibility(self_dict=dict1, reference_dict=dict2)
    compatibility(self_dict=dict2, reference_dict=dict1)

# Algorithm

Next, please find below my implementation of the *man-proposing algorithm*.

In [15]:
def MatchingGameSolution(men_pref, women_pref, print_on=False):
    # Validation rules
    validate_inputs(men_pref, women_pref)

    ###########################
    # Man-proposing algorithm
    ###########################
    # dictionary for storing engaged pairs
        # key: girl
        # value: boy
    engaged_girls = {}
    
    # list of not engaged guys
    proposing_guys = list(men_pref.keys())

    while proposing_guys:
        # choose a guy
        he = proposing_guys.pop(0)
        # list of girls he lieks
        girls_he_likes = men_pref[he]

        if print_on:
            print(f'he={he} likes girls={girls_he_likes}:')
        
        # propose to the girls in his list
        for she in girls_he_likes:

            #############################################
            # Preparation - answering questions
            #############################################

            # who is she engaged to?
            boy_shes_engaged_to = engaged_girls.get(she)
            # is she engaged?
            engaged = boy_shes_engaged_to is not None

            if engaged:
                # what boys does she like?
                boys_she_likes = women_pref[she]
                # current_partner rank
                current_partner = boys_she_likes.index(boy_shes_engaged_to)
                # new guy rank
                new_guy = boys_she_likes.index(he)
                # compare guys (smaller rank is preferred)
                she_likes_him_better = new_guy < current_partner

            #############################################
            # Case1) Not engaged
            #############################################
            if not engaged:
                if print_on:
                    print(f'  Case1) she={she} is not engaged and marries him={he}. :-)')
                # if she's not engaged than she marries the proposing man
                engaged_girls.update({she: he})
                break

            #############################################
            # Case2) Engaged, breaks up with current partner
            #############################################
            elif she_likes_him_better:
                if print_on:
                    print(f'  Case2) she={she} is engagaged to him={boy_shes_engaged_to}, but likes him={he} better. :-)')
                proposing_guys.insert(0, boy_shes_engaged_to)
                engaged_girls.update({she: he})
                break

            #############################################
            # Case3) Engaged, stays with current partner
            #############################################
            else:
                if print_on:
                    print(f'  Case3) she={she} is engagaged to him={boy_shes_engaged_to}, so he={he} goes on. :-(')
    return [(he, she) for she, he in engaged_girls.items()]

# Matching game for bipartite graphs

In [16]:
men_pref = {0: [5,6,7], 1: [6,4,7], 2: [5,7,4], 3: [6,5,4]}
women_pref = {4: [1,2,3], 5: [2,3,0], 6: [1,3,0], 7: [2,0,1]}

In [17]:
MatchingGameSolution(men_pref, women_pref, print_on=True)

he=0 likes girls=[5, 6, 7]:
  Case1) she=5 is not engaged and marries him=0. :-)
he=1 likes girls=[6, 4, 7]:
  Case1) she=6 is not engaged and marries him=1. :-)
he=2 likes girls=[5, 7, 4]:
  Case2) she=5 is engagaged to him=0, but likes him=2 better. :-)
he=0 likes girls=[5, 6, 7]:
  Case3) she=5 is engagaged to him=2, so he=0 goes on. :-(
  Case3) she=6 is engagaged to him=1, so he=0 goes on. :-(
  Case1) she=7 is not engaged and marries him=0. :-)
he=3 likes girls=[6, 5, 4]:
  Case3) she=6 is engagaged to him=1, so he=3 goes on. :-(
  Case3) she=5 is engagaged to him=2, so he=3 goes on. :-(
  Case1) she=4 is not engaged and marries him=3. :-)


[(2, 5), (1, 6), (0, 7), (3, 4)]